In [8]:
import pandas as pd
from pynif import NIFCollection
import requests
import time

In [9]:
s = ""
with open ('./training/oke17task1Training.xml.ttl') as f:
    s = f.read()
oke_task1_col = NIFCollection.loads(s, format='turtle')

for context in oke_task1_col.contexts[:2]:
    print(context.mention)
    for phrase in context.phrases:
        print(f'\t{phrase}')

Two friends in their early 20s ( Hill and Teller ) living in Miami Beach during the Iraq War exploit a little-known government initiative that allows small businesses to bid on U.S. Military contracts. Starting small, they begin raking in big money and are living the high life. 
	<NIFPhrase 61-72: 'Miami Beach'>
	<NIFPhrase 42-48: 'Teller'>
	<NIFPhrase 33-37: 'Hill'>
When Daniel Ocean is released from prison in New Jersey, his next heist is already planned. Danny's target are three Las Vegas casinos : The Bellagio, the Mirage and the MGM Grand. They all belong to ruthless entrepreneur Terry Benedict, who, by the way, also shows a certain interest in Danny's beautiful ex-wife Tess. 
	<NIFPhrase 117-134: 'Las Vegas casinos'>
	<NIFPhrase 5-17: 'Daniel Ocean'>
	<NIFPhrase 92-99: "Danny's">
	<NIFPhrase 151-161: 'the Mirage'>
	<NIFPhrase 222-236: 'Terry Benedict'>
	<NIFPhrase 45-55: 'New Jersey'>
	<NIFPhrase 137-149: 'The Bellagio'>
	<NIFPhrase 288-295: "Danny's">
	<NIFPhrase 314-318: 'Tess'

In [10]:
s = ""
with open ('./training/oke17task2Training.xml.ttl') as f:
    s = f.read()
    
oke_task2_col = NIFCollection.loads(s, format='turtle')

for context in oke_task2_col.contexts[:2]:
    print(context.mention)
    for phrase in context.phrases:
        print(f'\t{phrase}')

On the day that Hurricane Katrina hits New Orleans, elderly Daisy Williams (nee Fuller) is on her deathbed in a New Orleans hospital. At her side is her adult daughter, Caroline. Daisy asks Caroline to read to her aloud the diary of Daisy's lifelong friend, Benjamin Button. Benjamin's diary recounts his entire extraordinary life, the primary unusual aspect of which was his aging backwards, being diagnosed with several aging diseases at birth and thus given little chance of survival, but who does survive and gets younger with time. 
	<NIFPhrase 39-50: 'New Orleans'>
	<NIFPhrase 258-273: 'Benjamin Button'>
	<NIFPhrase 112-132: 'New Orleans hospital'>
	<NIFPhrase 179-184: 'Daisy'>
	<NIFPhrase 169-177: 'Caroline'>
	<NIFPhrase 60-87: 'Daisy Williams (nee Fuller)'>
	<NIFPhrase 190-198: 'Caroline'>
Emma Stone sure knows the best way around a hackneyed red carpet question. The “La La Land” star stopped to speak with Ryan Seacrest on the red carpet ahead of Sunday’s Golden Globes and was, of co

In [11]:
def get_scores(spots, ground_truth):
#     spots = sorted(spots, key=lambda s: s['offset'])
#     ground_truth = sorted(ground_truth, key=lambda s: s.beginIndex)
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for spot in spots:
        matching = [x for x in ground_truth if x.beginIndex == spot['offset']]
        if not matching:
            false_positives += 1
            continue
        matching = matching[0]
        if spot['uri'] != matching.taIdentRef:
            false_positives += 1
            continue
        true_positives += 1
    for true_spot in ground_truth:
        matching = [x for x in spots if true_spot.beginIndex == x['offset']]
        if not matching:
            false_negatives += 1
#     print(f'tp: {true_positives} fp: {false_positives} fn: {false_negatives}')
    
    precision = 0
    recall = 0
    f1 = 0
    
    if true_positives + false_positives + false_negatives == 0:
        # Empty gold standard. Set all as 1
        precision = 1
        recall = 1
    
    if true_positives + false_positives != 0:
        precision = true_positives / (true_positives + false_positives)
    if true_positives + false_negatives != 0:
        recall = true_positives / (true_positives + false_negatives)
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    
    return {
        'tp': true_positives,
        'fp': false_positives,
        'fn': false_negatives,
        'macro_precision': precision,
        'recall': recall,
        'f1': f1
    }

In [12]:
def _test_context(context, service_url, filter_policy):
    body = {
        'text': context.mention,
        'filter_policy': filter_policy
    }
    spots = requests.post(service_url, json=body).json()
    scores = get_scores(spots['data'], context.phrases)
    
    return scores

def test_collection(collection, service_url, filter_policy = 'NONE'):
    results = []
    for context in collection.contexts:
        result = _test_context(context, service_url, filter_policy)
        result['uri'] = context.uri
        results.append(result)
        time.sleep(0.5)
    return pd.DataFrame(results)

In [13]:
scores_spotlight = test_collection(oke_task1_col, 'http://localhost:4010/annotate')
print(scores_spotlight.head())

scores_spotlight2 = test_collection(oke_task2_col, 'http://localhost:4010/annotate')
print(scores_spotlight2.head())

   tp  fp  fn  macro_precision    recall        f1  \
0   0   2   2         0.000000  0.000000  0.000000   
1   1   4   9         0.200000  0.100000  0.133333   
2   4   2   0         0.666667  1.000000  0.800000   
3   6   2   1         0.750000  0.857143  0.800000   
4   4   6   2         0.400000  0.666667  0.500000   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  macro_precision    recall        f1  \
0   1   7   2         0.125000  0.333333  0.181818   
1   3   3   2         0.500000  0.600000  0.545455   
2   6   2   5         0.750000  0.545455  0.631579   
3   0  15   0         0.000000  0.000000  0.000000   
4   2   5   3         0.285714  0.400000  0.333333   

                    

In [14]:
scores_fox = test_collection(oke_task1_col, 'http://localhost:4020/annotate')
print(scores_fox.head())

scores_fox2 = test_collection(oke_task2_col, 'http://localhost:4020/annotate')
print(scores_fox2.head())

   tp  fp  fn  macro_precision    recall        f1  \
0   0   0   3         0.000000  0.000000  0.000000   
1   2   7   3         0.222222  0.400000  0.285714   
2   4   0   2         1.000000  0.666667  0.800000   
3   1   7   0         0.125000  1.000000  0.222222   
4   2   2   2         0.500000  0.500000  0.500000   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  macro_precision    recall        f1  \
0   1   8   0         0.111111  1.000000  0.200000   
1   2   1   3         0.666667  0.400000  0.500000   
2   4   4   4         0.500000  0.500000  0.500000   
3   3   0   3         1.000000  0.500000  0.666667   
4   4   3   2         0.571429  0.666667  0.615385   

                    

In [16]:
scores_tagme = test_collection(oke_task1_col, 'http://localhost:4030/annotate')
print(scores_tagme.head())

scores_tagme2 = test_collection(oke_task2_col, 'http://localhost:4030/annotate')
print(scores_tagme2.head())

   tp  fp  fn  macro_precision    recall        f1  \
0   0   1   3         0.000000  0.000000  0.000000   
1   2   3   7         0.400000  0.222222  0.285714   
2   5   1   0         0.833333  1.000000  0.909091   
3   1   3   5         0.250000  0.166667  0.200000   
4   3   3   2         0.500000  0.600000  0.545455   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  macro_precision    recall        f1  \
0   1   1   6             0.50  0.142857  0.222222   
1   2   2   3             0.50  0.400000  0.444444   
2   4   1   6             0.80  0.400000  0.533333   
3   0  10   4             0.00  0.000000  0.000000   
4   3   1   5             0.75  0.375000  0.500000   

                    

In [17]:
scores_all_none = test_collection(oke_task1_col, 'http://localhost:4040/annotate')
print(scores_all_none.head())

scores_all_none2 = test_collection(oke_task2_col, 'http://localhost:4040/annotate')
print(scores_all_none2.head())

   tp  fp  fn  macro_precision  recall        f1  \
0   0   4   1         0.000000    0.00  0.000000   
1   2   9   3         0.181818    0.40  0.250000   
2   4   2   0         0.666667    1.00  0.800000   
3   7   3   0         0.700000    1.00  0.823529   
4   3   9   1         0.250000    0.75  0.375000   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  macro_precision    recall        f1  \
0   1  10   0         0.090909  1.000000  0.166667   
1   3   4   2         0.428571  0.600000  0.500000   
2   5   5   4         0.500000  0.555556  0.526316   
3   0  17   0         0.000000  0.000000  0.000000   
4   4   4   2         0.500000  0.666667  0.571429   

                                

In [18]:
scores_all_r_singles = test_collection(oke_task1_col, 'http://localhost:4040/annotate', filter_policy='REMOVE_SINGLE_SPOTS')
print(scores_all_r_singles.head())

scores_all_r_singles2 = test_collection(oke_task2_col, 'http://localhost:4040/annotate', filter_policy='REMOVE_SINGLE_SPOTS')
print(scores_all_r_singles2.head())

   tp  fp  fn  macro_precision    recall        f1  \
0   0   2   2         0.000000  0.000000  0.000000   
1   2   3   7         0.400000  0.222222  0.285714   
2   4   2   0         0.666667  1.000000  0.800000   
3   7   0   1         1.000000  0.875000  0.933333   
4   3   2   2         0.600000  0.600000  0.600000   

                                                 uri  
0  http://www.ontologydesignpatterns.org/data/oke...  
1  http://www.ontologydesignpatterns.org/data/oke...  
2  http://www.ontologydesignpatterns.org/data/oke...  
3  http://www.ontologydesignpatterns.org/data/oke...  
4  http://www.ontologydesignpatterns.org/data/oke...  
   tp  fp  fn  macro_precision    recall        f1  \
0   1   6   2         0.142857  0.333333  0.200000   
1   2   1   3         0.666667  0.400000  0.500000   
2   5   1   5         0.833333  0.500000  0.625000   
3   0  10   2         0.000000  0.000000  0.000000   
4   4   3   2         0.571429  0.666667  0.615385   

                    

In [19]:
scores_spotlight.to_csv('scores/scores_spotlight_oke1.csv', index=False)
scores_fox.to_csv('scores/scores_fox_oke1.csv', index=False)
scores_tagme.to_csv('scores/scores_tagme_oke1.csv', index=False)
scores_all_none.to_csv('scores/scores_all_none_oke1.csv', index=False)
scores_all_r_singles.to_csv('scores/scores_all_r_singles_oke1.csv', index=False)

scores_spotlight2.to_csv('scores/scores_spotlight_oke2.csv', index=False)
scores_fox2.to_csv('scores/scores_fox_oke2.csv', index=False)
scores_tagme2.to_csv('scores/scores_tagme_oke2.csv', index=False)
scores_all_none2.to_csv('scores/scores_all_none_oke2.csv', index=False)
scores_all_r_singles2.to_csv('scores/scores_all_r_singles_oke2.csv', index=False)